In [1]:
import pandas as pd
import mlutils
from sklearn import svm, metrics
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from ipywidgets import widgets

# column names
feature_col = "features" # feature vector
value_col = "foldClass" # fold class to be predicted

# Read data
df = pd.read_json("./intermediate_data/features.json")

# Split data into train, and test data. 
# train: 90% test: 10%
train, test = train_test_split(df, test_size=0.1, random_state=13, stratify=df[value_col])
print("Training set size:", train.shape[0], "\n")
print(train[value_col].value_counts())
print()
print("Test set size:", test.shape[0], "\n")
print(test[value_col].value_counts())

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Training set size: 4833 

alpha         2529
alpha+beta    1519
beta           785
Name: foldClass, dtype: int64

Test set size: 537 

alpha         281
alpha+beta    169
beta           87
Name: foldClass, dtype: int64


In [2]:
import numpy as np
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# Convert the X into np.array

# train, val = train_test_split(train, test_size=0.1, random_state=13, stratify=train[value_col])

train_X = np.array(train[feature_col].tolist())
train_y = train[value_col]
# val_X = np.array(val[feature_col].tolist())
# val_y = val[value_col]
test_X = np.array(test[feature_col].tolist())
test_y = test[value_col]

# Hot-One-Encoding for LSTM's target values, y

def hot_one_encoding(values):
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded

train_y_num = hot_one_encoding(train_y)
test_y_num = hot_one_encoding(test_y)

# Reshape input values, X, for LSTM model

train_X_num = train_X.reshape((train_X.shape[0], train_X.shape[1], 1))
test_X_num = test_X.reshape((test_X.shape[0], test_X.shape[1], 1))

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [5]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.losses import categorical_crossentropy

model = Sequential()
model.add(LSTM(5, input_shape=(100,1)))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_X_num, train_y_num, batch_size=32, epochs=5)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
4833/4833 [==============================] - 29s 6ms/step - loss: 1.0273 - acc: 0.4827
Epoch 2/5
4833/4833 [==============================] - 29s 6ms/step - loss: 0.9925 - acc: 0.5233
Epoch 3/5
4833/4833 [==============================] - 28s 6ms/step - loss: 0.9881 - acc: 0.5233
Epoch 4/5
4833/4833 [==============================] - 29s 6ms/step - loss: 0.9843 - acc: 0.5233
Epoch 5/5
4833/4833 [==============================] - 29s 6ms/step - loss: 0.9802 - acc: 0.5233


In [6]:
loss, accuracy = model.evaluate(test_X_num, test_y_num)

print("Accuracy: {}".format(accuracy))

537/537 [==============================] - 1s 1ms/step
Accuracy: 0.523277468743493


In [11]:
# 2 hidden layers

model_2 = Sequential()
mod)
model_2.add(Activael_2.add(LSTM(5, return_sequences=True, input_shape=(100,1)))
model_2.add(LSTM(5))
model_2.add(Dense(3)tion('softmax'))

model_2.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

model_2.fit(train_X_num, train_y_num, batch_size=32, epochs=5)

loss_2, accuracy_2 = model_2.evaluate(test_X_num, test_y_num)

print("Accuracy: {}".format(accuracy_2))

# 3 hidden layers

model_3 = Sequential()
model_3.add(LSTM(5, return_sequences=True, input_shape=(100,1)))
model_3.add(LSTM(5))
model_3.add(LSTM(5))
model_3.add(Dense(3))
model_3.add(Activation('softmax'))

model_3.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

model_3.fit(train_X_num, train_y_num, batch_size=32, epochs=5)

loss_3, accuracy_3 = model_3.evaluate(test_X_num, test_y_num)

print("Accuracy: {}".format(accuracy_3))

Epoch 1/5
4833/4833 [==============================] - 57s 12ms/step - loss: 1.0441 - acc: 0.5096
Epoch 2/5
4833/4833 [==============================] - 55s 11ms/step - loss: 0.9981 - acc: 0.5233
Epoch 3/5
4833/4833 [==============================] - 55s 11ms/step - loss: 0.9969 - acc: 0.5233
Epoch 4/5
4833/4833 [==============================] - 55s 11ms/step - loss: 0.9950 - acc: 0.5233
Epoch 5/5
537/537 [==============================] - 2s 3ms/step
Accuracy: 0.523277468743493


ValueError: Input 0 is incompatible with layer lstm_13: expected ndim=3, found ndim=2